In [10]:
print('hi')

hi


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [12]:
# Set parameters
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
img_width, img_height = 224, 224
batch_size = 32
epochs = 20
num_classes = 6
learning_rate = 0.0001

# Define the data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(img_width),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    #,
    #'val': transforms.Compose([
     #   transforms.Resize(256),
     #   transforms.CenterCrop(img_width),
     #   transforms.ToTensor(),
     #   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    #]),
}



In [13]:
# Define the dataset directories
train_data_dir = '/newstorage5/aayesha/deep_learning/classification/fabric_classification/Fabrics_dataset'
#validation_data_dir = '/Users/aayeshaaayesha/Downloads/dtd/root/train'

# Load the datasets
train_dataset = datasets.ImageFolder(train_data_dir, transform=data_transforms['train'])
#val_dataset = datasets.ImageFolder(validation_data_dir, transform=data_transforms['val'])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the ResNet50 model pre-trained on ImageNet
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer to match the number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [14]:
# Training loop
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    #for phase in ['train', 'val']:
    for phase in ['train']:
        if phase == 'train':
            model.train()  # Set model to training mode
            dataloader = train_loader
        #else:
         #   model.eval()  # Set model to evaluate mode
          #  dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass and optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    print()

Epoch 1/20
----------
train Loss: 0.8769 Acc: 0.7552

Epoch 2/20
----------
train Loss: 0.2350 Acc: 0.9283

Epoch 3/20
----------
train Loss: 0.1603 Acc: 0.9528

Epoch 4/20
----------
train Loss: 0.1653 Acc: 0.9545

Epoch 5/20
----------
train Loss: 0.1516 Acc: 0.9510

Epoch 6/20
----------
train Loss: 0.1027 Acc: 0.9650

Epoch 7/20
----------
train Loss: 0.0852 Acc: 0.9790

Epoch 8/20
----------
train Loss: 0.1129 Acc: 0.9668

Epoch 9/20
----------
train Loss: 0.1077 Acc: 0.9633

Epoch 10/20
----------
train Loss: 0.1419 Acc: 0.9598

Epoch 11/20
----------
train Loss: 0.0794 Acc: 0.9773

Epoch 12/20
----------
train Loss: 0.0869 Acc: 0.9720

Epoch 13/20
----------
train Loss: 0.0813 Acc: 0.9738

Epoch 14/20
----------
train Loss: 0.0525 Acc: 0.9825

Epoch 15/20
----------
train Loss: 0.0709 Acc: 0.9720

Epoch 16/20
----------
train Loss: 0.0530 Acc: 0.9790

Epoch 17/20
----------
train Loss: 0.0615 Acc: 0.9790

Epoch 18/20
----------
train Loss: 0.0930 Acc: 0.9720

Epoch 19/20
-------

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'resnet50_custom_fabric_multiclass_model_20Epochs.pth')